## Scrape of Lyrics from Top Tracks of Top Artists

### Imports

In [3]:
import pandas as pd
import re
from multiprocessing import cpu_count


from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from webdriver_manager.firefox import GeckoDriverManager
from joblib import Parallel, delayed
from os.path import isfile

import time
import json
import warnings
warnings.filterwarnings('ignore')


#### Driver Options

In [4]:
PATH=FirefoxService(GeckoDriverManager().install())
binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\Firefox.exe')

opciones=Options()
#opciones.headless = True
#opciones.add_argument('--incognito')

[WDM] - Downloading: 16.9kB [00:00, 8.63MB/s]                   


In [5]:
with open('../data/tops/_paths.json', 'r') as fp: # Load paths
    artist_paths = json.load(fp)

In [10]:
artist_paths['Queen']

'../data/tops/Queen.parquet'

In [11]:
url = 'https://www.azlyrics.com/lyrics/' 
lyric_XPATH = '/html/body/div[2]/div[2]/div[2]/div[5]'
fail_dic = {}

def lyric_path(artist):
    return '../data/lyrics/' + artist + '.json'

def load_lyrics(artist):
    if isfile(lyric_path(artist)):
        with open(lyric_path(artist), 'r') as fp:
            total_unique_lyrics = json.load(fp)
    else:
        total_unique_lyrics = {'songs_considered' : [] , 'distinct_words': 0}
    return total_unique_lyrics

def initialize_dataframe(df,artist):
    if 'Unique_Word_Count' in df.columns:
        df = df[df['Unique_Word_Count'].isna()]
        if len(df) == 0 :
            print(artist, 'has already been fully uploaded')
            return None
    else:
        df['Unique_Word_Count'] = pd.NA
    return df , load_lyrics(artist)

# def remove_if_done(df,artist):
#     '''
#     Helper function for song_lyric_count.
    
#     Clears out dataframe rows for which the work has already been done
#     to avoid duplicate effort
#     '''


def song_lyric_count(artist):
    
    # top_hits = remove_if_done(pd.read_parquet(artists[artist]),artist)
    top_hits , total_unique_lyrics = initialize_dataframe(pd.read_parquet(artist_paths[artist]),artist)
    

    song_urls = []
    
    
    for ind,track in enumerate(top_hits['Track_Name']):
        song_urls.append((url + re.sub('[^A-Za-z0-9]+', '',  artist).lower() + '/' + re.sub('[^A-Za-z0-9]+', '', track).lower() + '.html' , track, ind ))
    
    fail_counter = 0
    for i in song_urls:
        driver = webdriver.Firefox(firefox_binary=binary,options = opciones)
        try:
            driver.get(i[0])
            time.sleep(2)
            lyrics = driver.find_element(By.XPATH, lyric_XPATH).text.split()
            unique_lyrics = set()
            for k in lyrics:
                j = re.sub('\'s','',k)
                j = re.sub('[^A-Za-z0-9]+', '', j)
                unique_lyrics.add(j)
            total_unique_lyrics[artist] = total_unique_lyrics[artist].union(unique_lyrics)
            top_hits.at[i[2],'Unique_Word_Count'] = len(unique_lyrics)
            print('Uploaded: ',artist, i[1] )
            driver.quit()
        except:
            fail_dic[artist] = i[1]
            fail_counter +=1
            print('Search failed: ',artist, i[1] )
            driver.quit()
    top_hits.to_parquet(artist_paths[artist],index=False)
    total_unique_lyrics[artist] = [len(total_unique_lyrics[artist]),total_unique_lyrics[artist]]
    print('Uploaded', artist, 'with',fail_counter, 'fails')
    return

In [25]:
song_lyric_count('Queen'])

passdriver
pass lyrics
pass totaluniquelyrics
pass yat
passdriver
pass lyrics
pass totaluniquelyrics
pass yat
passdriver
pass lyrics
pass totaluniquelyrics
pass yat
passdriver
pass lyrics
pass totaluniquelyrics
pass yat
passdriver
pass lyrics
pass totaluniquelyrics
pass yat
passdriver
pass lyrics
pass totaluniquelyrics
pass yat


In [5]:
def main(cpus):
    Parallel(n_jobs=cpus, verbose=False,prefer="threads")(delayed(song_lyric_count)(i) for i in artist_paths)

In [7]:
main(max(cpu_count()-4,2))

Uploaded:  Rihanna Umbrella
Uploaded:  Madonna Hung Up
Uploaded:  Elvis_Presley Can't Help Falling in Love
Uploaded:  Michael_Jackson Billie Jean
Uploaded:  Queen Bohemian Rhapsody
Uploaded:  Led_Zeppelin Stairway to Heaven
Uploaded:  Elton_John Your Song
Search failed:  The_Beatles Come Together
Uploaded:  Rihanna Don't Stop the Music
Uploaded:  Madonna Like a Prayer
Uploaded:  Michael_Jackson Beat It
Uploaded:  Led_Zeppelin Immigrant Song
Uploaded:  Queen Another One Bites the Dust
Uploaded:  Elton_John Tiny Dancer
Uploaded:  Elvis_Presley Jailhouse Rock
Search failed:  The_Beatles Let It Be
Uploaded:  Rihanna Disturbia
Uploaded:  Madonna Material Girl
Uploaded:  Led_Zeppelin Whole Lotta Love
Search failed:  Elvis_Presley Suspicious Minds
Uploaded:  Queen Don't Stop Me Now
Uploaded:  Michael_Jackson Thriller
Uploaded:  Elton_John I'm Still Standing
Search failed:  The_Beatles Eleanor Rigby
Search failed:  The_Beatles Yesterday
Search failed:  Elvis_Presley Hound Dog
Search failed:  R

In [1]:
y

NameError: name 'y' is not defined

In [11]:
y = pd.read_parquet(artist_paths['Queen'])
z = y.drop(columns='Unique_Word_Count')

In [14]:
'Unique_Word_Count' in y.columns

True

In [22]:
z = y[y['Unique_Word_Count'].notna()]

In [19]:
x

,Rank,Track_Name,Listeners,Artist,Unique_Word_Count
8,9,Bohemian Rhapsody - Remastered 2011,814253,Queen,NaN


In [23]:
z

,Rank,Track_Name,Listeners,Artist,Unique_Word_Count
0,1,Bohemian Rhapsody,1650068,Queen,174.0
1,2,Another One Bites the Dust,1174505,Queen,109.0
2,3,Don't Stop Me Now,1168765,Queen,123.0
3,4,We Will Rock You,1084917,Queen,61.0
4,5,Killer Queen,1001659,Queen,126.0
5,6,We Are the Champions,914255,Queen,82.0
6,7,I Want to Break Free,880033,Queen,72.0
7,8,Somebody to Love,844672,Queen,131.0
9,10,Crazy Little Thing Called Love,813001,Queen,77.0


In [7]:
y = pd.read_parquet(artist_paths['Elton_John'])
y

,Rank,Track_Name,Listeners,Artist,Unique_Word_Count
0,1,Your Song,963403,Elton_John,131.0
1,2,Tiny Dancer,933266,Elton_John,104.0
2,3,I'm Still Standing,774812,Elton_John,105.0
3,4,"Rocket Man (I Think It's Going to Be a Long, L...",597714,Elton_John,NaN
4,5,Crocodile Rock,520580,Elton_John,NaN
5,6,Don't Go Breaking My Heart,515110,Elton_John,NaN
6,7,Cold Heart - PNAU Remix,474734,Elton_John,NaN
7,8,Step Into Christmas,467852,Elton_John,NaN
8,9,Goodbye Yellow Brick Road,415806,Elton_John,NaN
9,10,Can You Feel the Love Tonight,409478,Elton_John,NaN


In [8]:
total_unique_lyrics

{}